In [1]:
import joblib
import numpy as np
import pandas as pd

import matplotlib.pyplot as plt
import seaborn as sns

import os

Relative imports are confusing, and I only have three functions anyway.

In [2]:
def postcodes_to_latlong(df: pd.DataFrame):
    spatial = pd.read_excel("../initial_docs/inspost.xls")
    spatial.drop(columns=["INS", "COMMUNE"], inplace=True)  # we do not need these info
    tmp_df = pd.merge(df, spatial, how="left", on="CODPOSS")
    tmp_df.drop(columns="CODPOSS", inplace=True)
    
    return tmp_df


def cat_to_numeric(data: pd.DataFrame):
    tmp = data.copy(deep=True)
    
    tmp['agecar'] = tmp['agecar'].map({"0-1": 0, "2-5": 1, "6-10":2, ">10": 3})
    tmp['sexp'] = tmp['sexp'].map({"Male": 0, "Female": 1})
    tmp['fuelc'] = tmp['fuelc'].map({"Petrol": 0, "Gasoil": 1})
    tmp['split'] = tmp['split'].map({"Once": 0, "Monthly": 1, "Twice":2, "Thrice": 3})
    tmp['usec'] = tmp['usec'].map({"Private": 0, "Professional": 1})
    tmp['fleetc'] = tmp['fleetc'].map({"Yes": 0, "No": 1})
    tmp['sportc'] = tmp['sportc'].map({"Yes": 0, "No": 1})
    tmp['coverp'] = tmp['coverp'].map({"MTPL": 0, "MTPL+": 1, "MTPL+++":2})
    tmp['powerc'] = tmp['powerc'].map({"<66": 0, "66-110": 1, ">110":2})
    
    return tmp


def avg_claim(row):
    if row['nbrtotc'] == 0:
        return 0
    else:
        return row['chargtot'] / row['nbrtotc']

Reading data and saved random forests.

In [5]:
test = pd.read_csv("../data/test.csv", index_col="Unnamed: 0")
test2 = postcodes_to_latlong(test)
test = cat_to_numeric(test2)

test.head()

,AGEPH,duree,lnexpo,nbrtotc,nbrtotan,chargtot,agecar,sexp,fuelc,split,usec,fleetc,sportc,coverp,powerc,LAT,LONG
0,55,0.495890,-0.701400,0,0.0,0.000000,1,0,1,0,0,1,1,2,0,50.466667,4.183333
1,46,1.000000,0.000000,1,1.0,145.017712,2,0,0,2,0,1,1,1,0,50.383333,4.683333
2,42,0.509589,-0.674151,0,0.0,0.000000,2,0,0,0,1,1,1,0,0,50.883333,3.433333
3,24,1.000000,0.000000,0,0.0,0.000000,1,0,0,3,0,1,1,0,0,51.216667,2.900000
4,79,1.000000,0.000000,0,0.0,0.000000,3,1,0,3,0,1,1,0,0,50.683333,4.200000


In [11]:
freq_rf = joblib.load("../models/freq_rf.pkl")
sev_rf = joblib.load("../models/sev_rf.pkl")

features = freq_rf_final.feature_names_in_  # needed features will be here

# Analyzing the tariff-plan

## Grabbing random forest predictions

In [12]:
test_freq_preds = freq_rf_final.predict(test.loc[:, features])
test_sev_preds = sev_rf_final.predict(test.loc[:, features])

In [17]:
test_results = test.loc[:, ["nbrtotc", "chargtot"]]  # to get zeroes first
test_results['avg_claim'] = test_results.apply(lambda row: avg_claim(row), axis=1)

test_results['rf_freq_preds'] = test_freq_preds
test_results['rf_sev_preds'] = test_sev_preds
test_results['rf_tariff_preds'] = test_results['rf_freq_preds']*test_results['rf_sev_preds']

test_results = test_results.iloc[:, [0, 2, 1, 3, 4, 5]]
test_results.query("nbrtotc > 1")[0:6]

,nbrtotc,avg_claim,chargtot,rf_freq_preds,rf_sev_preds,rf_tariff_preds
24,2,883.975915,1767.951829,0.168462,1466.489643,247.047585
29,2,278.582743,557.165486,0.166913,1734.311468,289.479941
69,2,158.676645,317.353290,0.163527,1963.060437,321.012661
126,2,384.495252,768.990503,0.152646,1229.984087,187.751719
154,2,840.842441,1681.684883,0.162941,1665.337240,271.352438
261,2,126.004279,252.008557,0.162606,1569.615446,255.229655


## Grabbing glm predictions